# Creating server using Express library

- This file explains how to create express server.
- You should use 12-express-client.ipynb to connect to this server as client.
- You should use both notebooks [server](12-express-server.ipynb) and [client](12-express-client.ipynb) along with each other.

**NOTE**: Last cell in this doc will stop the server. Only run it when you are finished.

In [1]:
const path = require('path');
const cwd = process.cwd();
const envPath = path.join(cwd, 'express-server-env', '.env');
require('dotenv').config({ path: envPath });
const mongoose = require('mongoose');
const bcrypt = require('bcryptjs');
const jwt = require('jsonwebtoken');
const express = require('express');

{
  parsed: {
    JWT_SECRET: 'e9c6a7c9e7b573a3a0b9dcb7c6f58d52e9f2b4d6b8f73aaf2e80bd99f46a2b0c'
  }
}

## Initiating Monggose database

In [2]:
const dbName = "nodejs-learners-package-express-db";
const dbPort = '27017'; // Set your MongoDB port
const uri = `mongodb://127.0.0.1:${dbPort}/${dbName}`; // Replace with your actual connection uri

In [3]:
const UserSchema = new mongoose.Schema({
    username: { type: String, required: true, unique: true },
    password: { type: String, required: true }
});

// Hash the password before saving
/*
Explanation:
UserSchema.pre('save', async (next) => {...}: This sets up a Mongoose middleware function that runs before 
    saving a document (pre-save hook). 
    The async keyword indicates that this function performs asynchronous operations.

if (!this.isModified('password')) return next();: Checks if the password field has been modified. 
    If the password hasn’t been changed, there’s no need to hash it again,
    so the middleware proceeds to the next function (next()). 
    This prevents unnecessary hashing if the document is being updated but the password hasn’t changed.

const salt = await bcrypt.genSalt(10);: Generates a salt using bcrypt. 
    A salt is a random value added to the password before hashing to ensure that 
    identical passwords have different hashes. 
    The 10 is the salt rounds, which determines the computational cost of the hashing. 
    More rounds mean more security but slower hashing.

this.password = await bcrypt.hash(this.password, salt);: Hashes the password with the generated salt. 
    The this.password refers to the password field of the current document. 
    The await keyword ensures that the function waits for the hashing to complete before proceeding.

next();: Moves on to the next middleware or completes the save operation if there are no more middlewares.
*/
UserSchema.pre('save', async (next) => {
    if (!this.isModified('password')) return next();
    const salt = await bcrypt.genSalt(10);
    this.password = await bcrypt.hash(this.password, salt);
    next();
});

// Method to compare passwords
/*
Explanation:
UserSchema.methods.comparePassword = (candidatePassword) => {...}: Adds a method to the user schema. 
    This method is an instance method, which means it operates on a specific user document 
    (i.e., `this` refers to the instance of the user document).

bcrypt.compare(candidatePassword, this.password): Uses bcrypt to compare the candidatePassword 
    (the plain text password provided by a user during login) with the hashed password stored in the 
    this.password field (the hashed password from the database). 
    The bcrypt.compare function returns a promise that resolves to true if the passwords match and false otherwise.
*/
UserSchema.methods.comparePassword = (candidatePassword) => {
    return bcrypt.compare(candidatePassword, this.password);
};

const User = mongoose.model('User', UserSchema);

[Function (anonymous)]

## Creating middlewares

In [4]:
const authMiddleware = (req, res, next) => {
    const token = req.headers['authorization']?.split(' ')[1];
    if (!token) return res.status(401).json({ message: 'No token provided' });

    jwt.verify(token, process.env.JWT_SECRET, (err, decoded) => {
        if (err) return res.status(403).json({ message: 'Invalid token' });
        req.user = decoded; // <-- Token payload is added to req 
        next();
    });
};

## Creating routers

In [5]:
const app = express();
const authRoutes = express.Router();

// Sign Up
authRoutes.post('/signup', async (req, res) => {
    try {
        const { username, password } = req.body;

        // Check for existing username
        const existingUser = await User.findOne({ username });

        if (existingUser) {
          return res.status(400).json({ message: 'Username already exists' });
        }

        const user = new User({ username, password });
        await user.save();   

        res.status(201).json({ message: 'User created' });
    } catch (error) {
        res.status(400).json({ message: error.message });
    }
});

// Sign In
authRoutes.post('/signin', async (req, res) => {
    try {
        const { username, password } = req.body;
        const user = await User.findOne({ username });
        if (!user || !(await user.comparePassword(password))) {
            return res.status(400).json({ message: 'Invalid credentials' });
        }

        const token = jwt.sign({ id: user._id }, process.env.JWT_SECRET, { expiresIn: '1h' });
        res.json({ token });
    } catch (error) {
        res.status(400).json({ message: error.message });
    }
});

const userRoutes = express.Router();
// Protected route example; uses authMiddleware for jwt authentication
userRoutes.get('/profile', authMiddleware, (req, res) => {
    res.json({ message: 'This is a protected route', user: req.user });
});

[Function: router] {
  params: {},
  _params: [],
  caseSensitive: undefined,
  mergeParams: undefined,
  strict: undefined,
  stack: [
    Layer {
      handle: [Function: bound dispatch],
      name: 'bound dispatch',
      params: undefined,
      path: undefined,
      keys: [],
      regexp: /^\/profile\/?$/i,
      route: [Route]
    }
  ]
}

## Initiating the server

In [6]:
app.use(express.json());

// Routes
app.use('/users', userRoutes); // ./profile
app.use('/auth', authRoutes); // ./signin and ./signup

let server

// Connect to MongoDB
mongoose.connect(uri)
    .then(() => { 
        console.log('MongoDB connected');
        //const PORT = process.env.PORT || 5678;
        const PORT = 5678;
        server = app.listen(PORT, () => console.log(`Server running on port ${PORT}`));
    })
    .catch(err => {
        console.error('An error happened during server initiation:', err);
        process.exit(1); // Optional: Exit the process if connection fails
    });


Promise { <pending> }

MongoDB connected
Server running on port 5678


## Stopping the server

In [7]:
mongoose.disconnect().then(() => {
        console.log('MongoDB connection closed');
        server?.close(() => {
            console.log('Express server closed');
            process.exit();
        });
    });


Promise { <pending> }

MongoDB connection closed
Express server closed
